# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

In [ ]:
/* I created a spreadsheet that listed all the tables and columns. 
I color codeded it by filling the column names the same across the tables.
It helped me to visually be able to see what columns each table had in common. 
I am going to attempt to push it through to my repo as I was not successful in attaching it here */

2. What is the datatype for women employees?

In [ ]:
--the data type for women employees is string
--data type code for women employess is 10

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
SELECT *
FROM series
WHERE supersector_code = 55 AND data_type_code = 10 AND industry_code = 55522110

--CES5552211010

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:
SELECT SUM (value) as TotalEmployees
FROM annual_2016

--2351408916

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 


In [ ]:
SELECT SUM(value) AS TotalWomenEmp2016
FROM annual_2016
WHERE series_id LIKE '%10'

--1125490

3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:
SELECT SUM(value) As TotalProdAndNsEmp
FROM annual_2016
WHERE series_id LIKE '%06'

--1263650

4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
SELECT AVG(value) AS AvgWeeklyHours
FROM january_2017
WHERE series_id LIKE '%07'

--36 hours

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:
SELECT SUM(value) AS TotalPayroll
FROM january_2017
WHERE series_id LIKE '%30' 
    OR series_id LIKE '%31'

--2906970.24

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
SELECT *
FROM industry
WHERE industry_code = 31336350
--industry with highest = Motor vehicle power train components

SELECT *
FROM industry
WHERE industry_code = 70713940
--industry with lowest = Fitness and recreational sports centers

7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
SELECT *
FROM industry
WHERE industry_code = 55524130
--industry with highest weekly payroll = Reinsurance carriers

SELECT *
FROM industry
WHERE industry_code = 70713950
--lowest weekly payroll = Bowling Centers

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
SELECT TOP 50 *
FROM annual_2016 AS a  
    LEFT JOIN series AS s
    ON a.series_id = s.series_id
ORDER BY id


--I chose LEFT JOIN because I want all data from annual_2016, but only what matches from series.

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *
SELECT TOP 50 *
FROM series AS s  
    JOIN datatype AS d  
    ON s.data_type_code = d.data_type_code
ORDER BY series_id

--I chose inner joim because I only wanted what had matching records in both tables. 

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *
SELECT TOP 50 *
FROM series AS s  
    JOIN industry AS i    
    ON s.industry_code = i.industry_code
ORDER BY series_id

--I chose inner joim because I only wanted what had matching records in both tables

## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:
SELECT j.series_id, s.industry_code, i.industry_name, j.value
FROM january_2017 AS j   
JOIN series AS s ON s.series_id = j.series_id
JOIN industry AS i ON s.industry_code = i.industry_code
WHERE j.value > (
    SELECT AVG(a.value)
    FROM annual_2016 AS a
    JOIN series AS se   
    ON a.series_id = se.series_id
    WHERE se.data_type_code = '82'
)

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
-- Optional CTE below
WITH avg2016 AS (
    SELECT AVG(a.value) AS avgValue
    FROM annual_2016 AS a    
    JOIN series AS s  
    ON a.series_id = s.series_id
    WHERE s.data_type_code = '82'
)
SELECT j.series_id, s.industry_code, i.industry_name, j.value
FROM january_2017 AS j   
JOIN series AS s 
    ON s.series_id = j.series_id
JOIN industry AS i 
    ON s.industry_code = i.industry_code
CROSS JOIN avg2016 AS a   
WHERE j.value > a.avgValue

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
SELECT a.value, a.year, a.period 
FROM annual_2016 AS a    
JOIN series AS s
    ON a.series_id = s.series_id
WHERE s.data_type_code = '30'

UNION

SELECT j.value, j.year, j.period   
FROM january_2017 AS j    
JOIN series AS s2
    ON j.series_id = s2.series_id
WHERE s2.data_type_code = '30'

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

Employees fared better in January 2017 vs annual 2016. 
Avg weekly earnings in 2016 = 523.22
Avg weekly earnings in 2017 = 527.58


2. In which industries did production and nonsupervisory employees fare better?

Total private and private service providing industry employees fared better in both 2016 and 2017

3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

Honestly, because I am so new to datasets, there wasn't anything I didn't have. I'm sure there are things that would have made the analysis easier, but I'm not familiar enough to know for sure. 